# IBM Capstone project

### Opening a New Shopping Mall in Bangalore

In [2]:
!pip install geopy
!pip install geocoder
!pip install folium

import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.io.json import json_normalize
from sklearn.metrics import silhouette_score

%matplotlib notebook

print('All libraries imported.')

All libraries imported.


In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore").text

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [51]:
print(soup.find('title'))

<title>List of neighbourhoods in Bangalore - Wikipedia</title>


In [6]:
mytable=soup.find_all('table',{'class':'wikitable sortable'})

In [7]:
ntot=len(mytable)

In [8]:
print(ntot)

8


In [71]:
x=(mytable[2].find_all('a'))

In [79]:
nbd=[]

In [80]:
for i in range(0, 8):  # Gets all the neighbourhoods
    lists =mytable [i].find_all('a')
    for list in lists:
        nbd.append(list.get('title'))# Gets the title of the neighbourhood
#        print(nbd)

In [84]:
df_bgl=pd.DataFrame(nbd,columns=['Neighbourhoods'])

In [167]:
df_bgl.shape

(145, 3)

In [85]:
df_bgl.head(5)

,Neighbourhoods
0,Bangalore Cantonment
1,British Raj
2,Richmond Town
3,Austin Town
4,"Murphy Town, Bangalore"


In [87]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates


In [88]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [90]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df_bgl["Neighbourhoods"].tolist() ]

In [92]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [93]:
# merge the coordinates into the original dataframe
df_bgl['Latitude'] = df_coords['Latitude']
df_bgl['Longitude'] = df_coords['Longitude']

In [94]:
df_bgl.head(5)

,Neighbourhoods,Latitude,Longitude
0,Bangalore Cantonment,12.97568,77.60539
1,British Raj,13.10509,77.60761
2,Richmond Town,12.96234,77.60123
3,Austin Town,12.96348,77.61297
4,"Murphy Town, Bangalore",12.98003,77.62776


In [96]:
# save the DataFrame as CSV file
df_bgl.to_csv("df_bgl.csv", index=False)

In [99]:
latitude  =12.9716
longitude =77.5946
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_bgl['Latitude'],df_bgl['Longitude'], df_bgl['Neighbourhoods']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [101]:

# save the map as HTML file
map_kl.save('map_bgl.html')

In [102]:
# define Foursquare Credentials and Version
CLIENT_ID = 'C2MYPOXYW4NZKBMNKYKQ1FD32VJLKPGLN5132IMVVPGPCWJL' # your Foursquare ID
CLIENT_SECRET = '4QO13LKMXP502EZVKZDKWP0HG34TKPFKNS45ENCCXX4R2QLW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C2MYPOXYW4NZKBMNKYKQ1FD32VJLKPGLN5132IMVVPGPCWJL
CLIENT_SECRET:4QO13LKMXP502EZVKZDKWP0HG34TKPFKNS45ENCCXX4R2QLW


In [103]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_bgl['Latitude'], df_bgl['Longitude'], df_bgl['Neighbourhoods']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [104]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhoods', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9946, 7)


,Neighbourhoods,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bangalore Cantonment,12.97568,77.60539,M.G Road Boulevard,12.975771,77.603979,Plaza
1,Bangalore Cantonment,12.97568,77.60539,Blossom Book House,12.975042,77.604813,Bookstore
2,Bangalore Cantonment,12.97568,77.60539,Brik Oven,12.974757,77.605441,Pizza Place
3,Bangalore Cantonment,12.97568,77.60539,Starbucks,12.974436,77.607308,Coffee Shop
4,Bangalore Cantonment,12.97568,77.60539,The 13th Floor,12.975364,77.604995,Lounge


In [105]:
venues_df.groupby(["Neighbourhoods"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhoods,,,,,,
100 Feet Road,92,92,92,92,92,92
Anjanapura,4,4,4,4,4,4
Anugraha,21,21,21,21,21,21
Arekere,82,82,82,82,82,82
Austin Town,100,100,100,100,100,100
...,...,...,...,...,...,...
"Whitefield, Bangalore",69,69,69,69,69,69
Wipro,61,61,61,61,61,61
Yelahanka,24,24,24,24,24,24


In [106]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 226 uniques categories.


In [107]:

# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Plaza', 'Bookstore', 'Pizza Place', 'Coffee Shop', 'Lounge',
       'American Restaurant', 'Pub', 'Toy / Game Store', 'Café',
       'Burger Joint', 'Indian Restaurant', 'Ice Cream Shop',
       'Andhra Restaurant', 'Music Store', 'Afghan Restaurant',
       'Breakfast Spot', 'Cricket Ground', 'Brewery', 'Deli / Bodega',
       'Bubble Tea Shop', 'Hotel', 'Italian Restaurant', 'Gym',
       'Sushi Restaurant', 'Shopping Mall', 'Park', 'Japanese Restaurant',
       'Cocktail Bar', 'French Restaurant', 'Furniture / Home Store',
       'Chinese Restaurant', 'Cupcake Shop', 'Soccer Stadium',
       'Dessert Shop', 'Gym / Fitness Center', 'Clothing Store',
       'Athletics & Sports', 'Bakery', 'Asian Restaurant', 'Market',
       'Korean Restaurant', 'Tea Room', 'Department Store',
       "Women's Store", 'Eastern European Restaurant',
       'Mediterranean Restaurant', 'Fried Chicken Joint',
       'Thai Restaurant', 'Bed & Breakfast', 'Steakhouse'], dtype=object)

In [108]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [142]:
# one hot encoding
bgl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bgl_onehot['Neighbuorhoods'] = venues_df['Neighbourhoods'] 

# move neighborhood column to the first column
cols = bgl_onehot.columns.values.tolist()
#print(cols)

cols2 = [cols[-1]] + cols[:-1]
#print(cols2)
bgl_onehot2 = bgl_onehot[cols2]
#bgl_onehot = bgl_onehot[fixed_columns]
#print(bgl_onehot.shape)
bgl_onehot2.head()
#print(len(cols2),cols.shape)

,Neighbuorhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,...,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:

bgl_grouped = bgl_onehot2.groupby(["Neighbuorhoods"]).mean().reset_index()

print(bgl_grouped.shape)
bgl_grouped

(110, 227)


,Neighbuorhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,...,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,100 Feet Road,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.021739,0.0,0.0,0.0,0.01087,0.0
1,Anjanapura,0.25,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
2,Anugraha,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
3,Arekere,0.00,0.0,0.000000,0.0,0.0,0.012195,0.012195,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.012195,0.0,0.0,0.0,0.00000,0.0
4,Austin Town,0.00,0.0,0.000000,0.0,0.0,0.010000,0.010000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,"Whitefield, Bangalore",0.00,0.0,0.000000,0.0,0.0,0.014493,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
106,Wipro,0.00,0.0,0.016393,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
107,Yelahanka,0.00,0.0,0.000000,0.0,0.0,0.041667,0.000000,0.0,0.0,...,0.041667,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.00000,0.0
108,Yeshwanthpur,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.133333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0


In [145]:
len(bgl_grouped[bgl_grouped["Shopping Mall"] > 0])

56

In [146]:
bgl_mall = bgl_grouped[["Neighbuorhoods","Shopping Mall"]]

In [147]:
bgl_mall.head(5)

,Neighbuorhoods,Shopping Mall
0,100 Feet Road,0.01087
1,Anjanapura,0.00000
2,Anugraha,0.00000
3,Arekere,0.02439
4,Austin Town,0.01000


In [150]:
bclusters = 3

bgl_clustering = bgl_mall.drop(["Neighbuorhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=bclusters, random_state=0).fit(bgl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0])

In [151]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bgl_merged = bgl_mall.copy()

# add clustering labels
bgl_merged["Cluster Labels"] = kmeans.labels_

In [157]:
bgl_merged.rename(columns={"Neighbourhood": "Neighbourhoods"}, inplace=True)
bgl_merged.head()

,Neighbourhoods,Shopping Mall,Cluster Labels
0,100 Feet Road,0.01087,0
1,Anjanapura,0.00000,0
2,Anugraha,0.00000,0
3,Arekere,0.02439,2
4,Austin Town,0.01000,0


In [158]:
df_bgl.head(5)

,Neighbourhoods,Latitude,Longitude
0,Bangalore Cantonment,12.97568,77.60539
1,British Raj,13.10509,77.60761
2,Richmond Town,12.96234,77.60123
3,Austin Town,12.96348,77.61297
4,"Murphy Town, Bangalore",12.98003,77.62776


In [159]:
bgl_merged=bgl_merged.join(df_bgl.set_index("Neighbourhoods"),on="Neighbourhoods")
print(bgl_merged.shape)
bgl_merged.head()

(110, 5)


,Neighbourhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,100 Feet Road,0.01087,0,12.906498,77.573195
1,Anjanapura,0.00000,0,12.858110,77.559090
2,Anugraha,0.00000,0,12.961200,77.510710
3,Arekere,0.02439,2,12.885680,77.596680
4,Austin Town,0.01000,0,12.963480,77.612970


In [160]:
# sort the results by Cluster Labels
print(bgl_merged.shape)
bgl_merged.sort_values(["Cluster Labels"], inplace=True)
bgl_merged

(110, 5)


,Neighbourhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,100 Feet Road,0.010870,0,12.906498,77.573195
73,Mathikere,0.000000,0,13.032350,77.558660
67,Madiwala,0.010000,0,12.920520,77.620900
64,Kumaraswamy Layout,0.000000,0,12.898190,77.559270
62,Kothnur,0.000000,0,13.064340,77.648550
...,...,...,...,...,...
33,"Fraser Town, Bangalore",0.022222,2,12.998940,77.612760
85,Padmanabhanagar,0.027397,2,12.918140,77.559400
86,Peenya,0.045455,2,13.031850,77.526790
12,Bangalore Development Authority,0.023256,2,13.015223,77.536948


In [162]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(bclusters)
ys = [i+x+(i*x)**2 for i in range(bclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bgl_merged['Latitude'], bgl_merged['Longitude'], bgl_merged['Neighbourhoods'], bgl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [163]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [164]:
bgl_merged.loc[bgl_merged['Cluster Labels'] == 0]

,Neighbourhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,100 Feet Road,0.01087,0,12.906498,77.573195
73,Mathikere,0.00000,0,13.032350,77.558660
67,Madiwala,0.01000,0,12.920520,77.620900
64,Kumaraswamy Layout,0.00000,0,12.898190,77.559270
62,Kothnur,0.00000,0,13.064340,77.648550
...,...,...,...,...,...
37,HBR Layout,0.00000,0,13.026420,77.624370
32,Embassy GolfLinks Business Park,0.00000,0,12.948660,77.645970
35,Global Village Tech Park,0.00000,0,12.919721,77.503531
30,Domlur,0.00000,0,12.943290,77.656020


In [165]:
bgl_merged.loc[bgl_merged['Cluster Labels'] == 1]

,Neighbourhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
50,Jalahalli,0.125,1,13.05450,77.52658
100,Varthur,0.080,1,12.94348,77.74703


In [166]:
bgl_merged.loc[bgl_merged['Cluster Labels'] == 2]

,Neighbourhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
10,Bangalore Cantonment,0.020000,2,12.975680,77.605390
93,Russell Market,0.020000,2,12.985070,77.605510
49,J. P. Nagar,0.020000,2,12.908310,77.590240
65,Lingarajapuram,0.029851,2,13.005480,77.625970
97,Ulsoor,0.020000,2,12.989080,77.627950
48,"International Tech Park, Bangalore",0.031746,2,12.985000,77.735900
3,Arekere,0.024390,2,12.885680,77.596680
105,"Whitefield, Bangalore",0.028986,2,12.979380,77.733720
106,Wipro,0.016393,2,12.911120,77.683780
44,Hulimavu,0.016129,2,12.880640,77.601470


Observations:
Most of the shopping malls are concentrated in the central area of Bangalore city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.Observations:
